imports

In [6]:
import numpy as np
import random
import pandas as pd 
from pylab import mpl, plt
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable

import vectorbtpro as vbt
from plotly.subplots import make_subplots
import plotly.graph_objects as go



data

In [3]:
sol_data = pd.read_csv('2ySOLdata1h.csv')
sol_data['timestamp'] = pd.to_datetime(sol_data['timestamp'], unit='s')
sol_data.set_index('timestamp', inplace=True)
pd.set_option('future.no_silent_downcasting', True)

In [4]:
data = sol_data.iloc[:, 0:5].copy()


In [5]:

data_trimmed = data.copy()
pd.set_option('future.no_silent_downcasting', True)
data_trimmed.loc[:, 'signal'] = 'SignalNone'


# Define window size
window_size = 10

rolling_max = data_trimmed.loc[:,'Close'].rolling(window=2*window_size+1, center=True, min_periods=1).max()
rolling_min = data_trimmed.loc[:,'Close'].rolling(window=2*window_size+1, center=True, min_periods=1).min()

is_peak = (data_trimmed.loc[:, 'Close'] == rolling_max)

is_low = (data_trimmed.loc[:, 'Close'] == rolling_min) 

# Update signal columns where conditions are met
data_trimmed.loc[is_peak, 'signal'] = 'SignalShort'  # Mark peaks with SignalShort
data_trimmed.loc[is_low, 'signal'] = 'SignalLong'   # Mark lows with SignalLong
df = data_trimmed.copy()
df_filtered = df[df['signal'] != 'SignalNone'].copy()

# Iterate through the DataFrame and adjust the signals
for i in range(1, len(df_filtered)):
    current_signal = df_filtered.iloc[i]['signal']
    previous_signal = df_filtered.iloc[i - 1]['signal']
    current_close = df_filtered.iloc[i]['Close']
    previous_close = df_filtered.iloc[i - 1]['Close']
    
    if current_signal == previous_signal:
        if current_signal == 'SignalLong' and previous_close > current_close:
            df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
        elif current_signal != 'SignalLong' and previous_close < current_close:
            df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
        else:
            df_filtered.iloc[i, df_filtered.columns.get_loc('signal')] = 'SignalNone'


df.update(df_filtered)

df.loc[:,'signal'] = df.loc[:,'signal'].replace({'SignalLong': 1, 'SignalShort': 0, 'SignalNone': 0.5})
df = df.ffill()

df['signal'] = df['signal'].astype(float)
long_signals = df['signal'] == 1
short_signals = df['signal'] == 0

scaler = MinMaxScaler(feature_range=(0, 1))
for idx in df.index[:-1]:
    if short_signals.loc[idx]:
        short_index = idx
        next_long_idx = df.loc[idx:].index[long_signals[idx:]].min()
        bear_slice = df.loc[short_index : next_long_idx].copy()
        bear_slice['signal'] = bear_slice['Close']
        signal_values = bear_slice['signal'].values.reshape(-1, 1)
        scaled_signal_values = scaler.fit_transform(signal_values)
        scaled_signal_values_transformed = 1 - (scaled_signal_values)
        bear_slice['signal'] = scaled_signal_values_transformed.flatten()
        df.update(bear_slice)
    elif long_signals.loc[idx]:
        long_index = idx
        next_short_idx = df.loc[idx:].index[short_signals[idx:]].min()
        bull_slice = df.loc[long_index : next_short_idx].copy()
        bull_slice['signal'] = bull_slice['Close']
        signal_values = bull_slice['signal'].values.reshape(-1, 1)
        scaled_signal_values = scaler.fit_transform(signal_values)
        scaled_signal_values_transformed = 1 - (scaled_signal_values)
        bull_slice['signal'] = scaled_signal_values_transformed.flatten()
        df.update(bull_slice)


KeyboardInterrupt: 

In [ ]:
df_sol=df.copy()

In [ ]:
df_sol = vbt.Data.from_data(df)

features = df_sol.run("talib", mavp=vbt.run_arg_dict(periods=14))

df_sol.data['symbol'] = pd.concat([df_sol.data['symbol'], features], axis=1)
# This will drop columns from the DataFrame where all values are NaN
df_sol.data['symbol'] = df_sol.data['symbol'].dropna(axis=1, how='all')


df_sol.data['symbol'] = df_sol.data['symbol'].dropna()
predictor_list = df_sol.data['symbol'].drop('signal', axis=1).columns.tolist()
X = df_sol.data['symbol'][predictor_list]

y = df_sol.data['symbol']['signal']

X.columns = X.columns.astype(str)


train test split

In [ ]:
# Split the data into a training set and a test set
# Assuming X is a DataFrame or a NumPy array
from sklearn.model_selection import train_test_split

indices = np.arange(X.shape[0])

# First, split your data into a training+validation set and a separate test set
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, indices, test_size=0.3, shuffle=False)


scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


sequence

In [ ]:
timestep = 40

def create_sequences(input_data, timestep):
    sequences = []
    data_len = len(input_data)
    for i in range(data_len - timestep):
        seq = input_data[i:(i + timestep)]
        sequences.append(seq)
    return np.array(sequences)

X_train_list = create_sequences(X_train_scaled, timestep)
X_test_list = create_sequences(X_test_scaled, timestep)
y_train_seq_ar = y_train[timestep:]
y_test_seq_ar = y_test[timestep:]

# Convert to numpy arrays
x_train_ar = np.array(X_train_list)
x_test_ar = np.array(X_test_list)  
y_train_seq = np.array(y_train_seq_ar).reshape(-1, 1)
y_test_seq = np.array(y_test_seq_ar).reshape(-1, 1)


# make training and test sets in torch
x_train_tensor = torch.from_numpy(x_train_ar).type(torch.Tensor)
x_test_tensor = torch.from_numpy(x_test_ar).type(torch.Tensor)
y_train_tensor = torch.from_numpy(y_train_seq).type(torch.Tensor)
y_test_tensor = torch.from_numpy(y_test_seq).type(torch.Tensor)
y_train_tensor.size(),x_train_tensor.size()

(torch.Size([12147, 1]), torch.Size([12147, 40, 178]))

model

In [ ]:
# Build model
#####################


# Here we define our model as a class
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers

        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 32, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out


input_dim = x_train_tensor.shape[2]
hidden_dim = 128
num_layers = 2
output_dim = 1
    
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
loss_fn = torch.nn.MSELoss()
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimiser, step_size=20, gamma=0.8)


In [ ]:
# Train model
#####################
num_epochs = 100
hist = np.zeros(num_epochs)  

model.train()
for t in range(num_epochs):
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    # model.hidden = model.init_hidden()
    
    # Forward pass
    y_train_pred = model(x_train_tensor)


    loss = loss_fn(y_train_pred, y_train_tensor)
    if t % 10 == 0:
        print("Epoch", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
    # Update parameters
    optimiser.step()
    scheduler.step()

Epoch 0 MSE:  0.3389410972595215
Epoch 10 MSE:  0.08426856249570847
Epoch 20 MSE:  0.08010318130254745
Epoch 30 MSE:  0.08002681285142899
Epoch 40 MSE:  0.07947637885808945
Epoch 50 MSE:  0.07841061800718307
Epoch 60 MSE:  0.07616624981164932
Epoch 70 MSE:  0.070908322930336
Epoch 80 MSE:  0.06189346686005592
Epoch 90 MSE:  0.05985812470316887


In [ ]:

with torch.no_grad():
    # make predictions
    model.eval()
    y_test_pred = model(x_test_tensor)


    y_train_pred_np = y_train_pred.detach().numpy().squeeze()
    y_test_pred_np = y_test_pred.detach().numpy().squeeze()

    print(y_train.shape)
    print(y_test.shape)
    print(y_train_pred_np.shape)
    print(y_test_pred_np.shape)

    # Calculate RMSE directly without inverse transformation
    trainScore = math.sqrt(mean_squared_error(y_train[timestep:], y_train_pred_np))
    print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(y_test[timestep:], y_test_pred_np))
    print('Test Score: %.2f RMSE' % (testScore))

(12187,)
(5224,)
(12147,)
(5184,)
Train Score: 0.24 RMSE
Test Score: 0.25 RMSE


plot

In [7]:
vbt.phelp(vbt.BBANDS.run)

BBANDS.run(
    close,
    window=Default(value=14),
    wtype=Default(value='simple'),
    alpha=Default(value=2),
    short_name='bb',
    hide_params=None,
    hide_default=True,
    **kwargs
):
    Run `BBANDS` indicator.
    
    * Inputs: `close`
    * Parameters: `window`, `wtype`, `alpha`
    * Outputs: `upper`, `middle`, `lower`
    * Lazy outputs: `percent_b`, `bandwidth`
    
    Pass a list of parameter names as `hide_params` to hide their column levels, or True to hide all.
    Set `hide_default` to False to show the column levels of the parameters with a default value.
    
    Other keyword arguments are passed to `BBANDS.run_pipeline`.


In [89]:
stdevup = 1.5
stdevdn = 1.8
vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 600
vbt.settings['plotting']['layout']['height'] = 300
signal = pd.Series(y_test[timestep:])
pred_sig = pd.Series(y_test_pred_np.flatten())
pred_sig.index = signal.index
combined_df = pd.concat([signal, pred_sig], axis=1) # 
bbands = vbt.talib("BBANDS").run(pred_sig, nbdevup=stdevup, nbdevdn=stdevdn)
bbands = bbands.dropna()
combined_df['upperband'] = bbands.upperband
combined_df['lowerband'] = bbands.lowerband
# combined_df = combined_df.ffill()
fig = combined_df.vbt.plot()
fig.update_layout(yaxis_title='signal')
fig.show()

In [90]:
# long_entries = pred_sig > 0.65
# long_exits = pred_sig < 0.28
long_entries = pred_sig > combined_df.upperband
long_exits = pred_sig < combined_df.lowerband
short_entries = pred_sig < 0.14
short_exits = pred_sig > 0.32
pf = vbt.Portfolio.from_signals(
    close=X_test.Close, 
    long_entries=long_entries, 
    long_exits=long_exits, 
    # short_entries=short_entries,
    # short_exits=short_exits,
    size=100,
    size_type='value',
    # accumulate=True,
    init_cash='auto'
)
pf.plot({"cum_returns", "orders"}, settings=dict(bm_returns=False)).show()


